In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
import pickle
import os
import NN

import dataset_utils
import importlib
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import LinearLocator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Conv3D, MaxPooling3D, Embedding, LSTM, Bidirectional, Reshape, Concatenate
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
import dataset_utils
from glob import glob

In [3]:
DATA_DIR = "data/entire_field_raw_3_class"
NUM_POINTS = 100
NUM_CLASSES = 3
BATCH_SIZE = 32
train_points, test_points, train_labels, test_labels, CLASS_MAP = dataset_utils.parse_dataset(NUM_POINTS, DATA_DIR)

processing class: jumping
processing class: speed_walking
processing class: walking


In [7]:
# Define the input shape: (sequence_length, num_features)
sequence_length = 100  # Example fixed sequence length
num_features = 5  # 5 input features (x pos, y pos, z pos, energy strength, speed)
num_classes = 3  # Number of classes for classification

# Define the LSTM model
model = models.Sequential()

# LSTM layer
model.add(layers.LSTM(64, input_shape=(sequence_length, num_features), return_sequences=False))

# Fully connected (Dense) layer
model.add(layers.Dense(32, activation='relu'))

# Output layer with Softmax activation for multi-class classification
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                17920     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 20099 (78.51 KB)
Trainable params: 20099 (78.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(train_points, train_labels, epochs=30, validation_data=(test_points, test_labels), verbose=1, callbacks=[early_stopping])

Epoch 1/30
45/45 [==============================] - 2s 37ms/step - loss: 0.4265 - accuracy: 0.8144 - val_loss: 0.5983 - val_accuracy: 0.7768
Epoch 2/30
45/45 [==============================] - 1s 24ms/step - loss: 0.4098 - accuracy: 0.8265 - val_loss: 0.5373 - val_accuracy: 0.7825
Epoch 3/30
45/45 [==============================] - 1s 24ms/step - loss: 0.4390 - accuracy: 0.8059 - val_loss: 0.6790 - val_accuracy: 0.7232
Epoch 4/30
45/45 [==============================] - 1s 25ms/step - loss: 0.5183 - accuracy: 0.7705 - val_loss: 0.5671 - val_accuracy: 0.7853
Epoch 5/30
45/45 [==============================] - 1s 26ms/step - loss: 0.4087 - accuracy: 0.8272 - val_loss: 0.5605 - val_accuracy: 0.7938


In [11]:
# Example training
model.fit(train_points, train_labels, epochs=20, batch_size=32)

Epoch 1/20
45/45 [==============================] - 1s 25ms/step - loss: 0.6306 - accuracy: 0.7146
Epoch 2/20
45/45 [==============================] - 1s 23ms/step - loss: 0.6353 - accuracy: 0.7203
Epoch 3/20
45/45 [==============================] - 1s 22ms/step - loss: 0.5518 - accuracy: 0.7613
Epoch 4/20
45/45 [==============================] - 1s 21ms/step - loss: 0.5430 - accuracy: 0.7564
Epoch 5/20
45/45 [==============================] - 1s 22ms/step - loss: 0.5421 - accuracy: 0.7712
Epoch 6/20
45/45 [==============================] - 1s 22ms/step - loss: 0.5574 - accuracy: 0.7472
Epoch 7/20
45/45 [==============================] - 1s 22ms/step - loss: 0.5401 - accuracy: 0.7698
Epoch 8/20
45/45 [==============================] - 1s 23ms/step - loss: 0.4960 - accuracy: 0.7897
Epoch 9/20
45/45 [==============================] - 1s 25ms/step - loss: 0.4957 - accuracy: 0.7826
Epoch 10/20
45/45 [==============================] - 1s 23ms/step - loss: 0.4715 - accuracy: 0.8024
Epoch 11/

In [14]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Make predictions on the test set
predictions = model.predict(test_points)

# Convert probabilities to predicted class labels (0, 1, or 2)
predicted_classes = np.argmax(predictions, axis=1)

# Compute the confusion matrix
conf_matrix = confusion_matrix(test_labels, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Classification report
class_report = classification_report(test_labels, predicted_classes)
print("Classification Report:")
print(class_report)

# Evaluate the model's accuracy
accuracy = accuracy_score(test_labels, predicted_classes)
print(f"Test Accuracy: {accuracy:.4f}")


12/12 [==============================] - 0s 10ms/step
Confusion Matrix:
[[ 92   0   3]
 [  3 176  11]
 [  9  47  13]]
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.97      0.92        95
           1       0.79      0.93      0.85       190
           2       0.48      0.19      0.27        69

    accuracy                           0.79       354
   macro avg       0.72      0.69      0.68       354
weighted avg       0.75      0.79      0.76       354

Test Accuracy: 0.7938
